In [267]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold

In [268]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [269]:
def train_and_test(t): # feed raw text in BERT so we will not preprocess
    
    sentence_maxlen = 0
    x_train = []
    y_train = []
    all_favor_tweets = []
    all_against_tweets = []
    
    with open(train_data_file, 'r') as trainfile:
        for line in trainfile:
            
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            if line[0].strip() != 'ID' and line[3].strip() == 'FAVOR' and line[1] == t:
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_favor_tweets.append(tweet)
            elif line[0].strip() != 'ID' and line[3].strip() == 'AGAINST' and line[1] == t:
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_against_tweets.append(tweet)
            
    x_train = all_favor_tweets + all_against_tweets
    y_train = np.append(np.ones(len(all_favor_tweets)), np.zeros(len(all_against_tweets))) 
    
    pp = len(x_train)
    x_test = []
    y_test = []
    all_favor_tweets_test = []
    all_against_tweets_test = []
    with open(test_data_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            if line[0] != 'ID' and line[3] == 'FAVOR' and line[1] == t:
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_favor_tweets_test.append(tweet)
            elif line[0].strip() != 'ID' and line[3].strip() == 'AGAINST' and line[1] == t:
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_against_tweets_test.append(tweet)

    x_test = all_favor_tweets_test + all_against_tweets_test
    y_test = np.append(np.ones(len(all_favor_tweets_test)), np.zeros(len(all_against_tweets_test)))

    
    return x_train, y_train, x_test, y_test, sentence_maxlen, pp

In [270]:
train_data_file = '/home/parush/stance/Experiments/stance_mohammed/train.txt'
test_data_file = '/home/parush/stance/Experiments/stance_mohammed/test.txt'
TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]

df = pd.read_csv(train_data_file, sep='\t')
df1 = pd.read_csv(test_data_file, sep='\t')
t = TARGETS[2]

In [271]:
# train_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [272]:
# train_data_file = '/home/parush/stance/Experiments/Data_MPCHI/train.txt'
# test_data_file = '/home/parush/stance/Experiments/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')
# 

In [273]:
train_texts, train_labels, test_texts, test_labels, sen_maxlen, pp = train_and_test(t)

In [274]:
# Bert Classifier
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts)) #cannot be more than 512
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

In [275]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=128, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=128, truncating="post", padding="post", dtype="int")

train_y = np.array(train_labels) == 1
test_y = np.array(test_labels) == 1

In [276]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [277]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [278]:
BATCH_SIZE = 32 # you can change it
EPOCHS = 50 # do not train for too many epochs as a large model like this will surely overfit and overflow the ram and disk
learning_rate = 2e-5 # you can decrease it more like 3e-5 but do not increase it more than 1e-5

In [279]:
'''
This is how you read data into Pytorch

'''
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [280]:
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=learning_rate)

In [281]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_texts) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  1
0/16.8125 loss: 0.7226166725158691 
Epoch:  1
1/16.8125 loss: 0.7084046900272369 
Epoch:  1
2/16.8125 loss: 0.6905917723973592 
Epoch:  1
3/16.8125 loss: 0.6734533905982971 
Epoch:  1
4/16.8125 loss: 0.6603018760681152 
Epoch:  1
5/16.8125 loss: 0.6631015539169312 
Epoch:  1
6/16.8125 loss: 0.6854724713734218 
Epoch:  1
7/16.8125 loss: 0.6815416514873505 
Epoch:  1
8/16.8125 loss: 0.6725233859486051 
Epoch:  1
9/16.8125 loss: 0.6732909500598907 
Epoch:  1
10/16.8125 loss: 0.67434669082815 
Epoch:  1
11/16.8125 loss: 0.6653572221597036 
Epoch:  1
12/16.8125 loss: 0.6636245663349445 
Epoch:  1
13/16.8125 loss: 0.6661753697054726 
Epoch:  1
14/16.8125 loss: 0.6684948603312174 
Epoch:  1
15/16.8125 loss: 0.6701503284275532 
Epoch:  1
16/16.8125 loss: 0.672521875185125 
Epoch:  2
0/16.8125 loss: 0.7038133144378662 
Epoch:  2
1/16.8125 loss: 0.6848434507846832 
Epoch:  2
2/16.8125 loss: 0.6613191564877828 
Epoch:  2
3/16.8125 loss: 0.6689949184656143 
Epoch:  2
4/16.8125 loss: 0.67

Epoch:  11
5/16.8125 loss: 0.01942917937412858 
Epoch:  11
6/16.8125 loss: 0.018306035548448563 
Epoch:  11
7/16.8125 loss: 0.01750418997835368 
Epoch:  11
8/16.8125 loss: 0.02500794983158509 
Epoch:  11
9/16.8125 loss: 0.02388704353943467 
Epoch:  11
10/16.8125 loss: 0.022592080503024838 
Epoch:  11
11/16.8125 loss: 0.021826599957421422 
Epoch:  11
12/16.8125 loss: 0.021372513057520755 
Epoch:  11
13/16.8125 loss: 0.031027966245476688 
Epoch:  11
14/16.8125 loss: 0.029642668676873048 
Epoch:  11
15/16.8125 loss: 0.028995387081522495 
Epoch:  11
16/16.8125 loss: 0.028163699104505426 
Epoch:  12
0/16.8125 loss: 0.021810971200466156 
Epoch:  12
1/16.8125 loss: 0.014526840532198548 
Epoch:  12
2/16.8125 loss: 0.015512933488935232 
Epoch:  12
3/16.8125 loss: 0.014035990345291793 
Epoch:  12
4/16.8125 loss: 0.013175446446985007 
Epoch:  12
5/16.8125 loss: 0.01231213465022544 
Epoch:  12
6/16.8125 loss: 0.011536497888820512 
Epoch:  12
7/16.8125 loss: 0.011172746773809195 
Epoch:  12
8/16.81

Epoch:  21
2/16.8125 loss: 0.003500269182647268 
Epoch:  21
3/16.8125 loss: 0.003448602801654488 
Epoch:  21
4/16.8125 loss: 0.01009344165213406 
Epoch:  21
5/16.8125 loss: 0.009177699801512063 
Epoch:  21
6/16.8125 loss: 0.010313426643343908 
Epoch:  21
7/16.8125 loss: 0.02442680750391446 
Epoch:  21
8/16.8125 loss: 0.022236387426447537 
Epoch:  21
9/16.8125 loss: 0.020524108200334013 
Epoch:  21
10/16.8125 loss: 0.018892734544351697 
Epoch:  21
11/16.8125 loss: 0.018403335144588102 
Epoch:  21
12/16.8125 loss: 0.017264886723401453 
Epoch:  21
13/16.8125 loss: 0.0163341533791806 
Epoch:  21
14/16.8125 loss: 0.015498943983887632 
Epoch:  21
15/16.8125 loss: 0.014782163532800041 
Epoch:  21
16/16.8125 loss: 0.014216104316908647 
Epoch:  22
0/16.8125 loss: 0.002517560264095664 
Epoch:  22
1/16.8125 loss: 0.0030064332531765103 
Epoch:  22
2/16.8125 loss: 0.018017044368510444 
Epoch:  22
3/16.8125 loss: 0.014162712555844337 
Epoch:  22
4/16.8125 loss: 0.011849951930344105 
Epoch:  22
5/16.

Epoch:  30
16/16.8125 loss: 0.015697878169114974 
Epoch:  31
0/16.8125 loss: 0.003362893359735608 
Epoch:  31
1/16.8125 loss: 0.00423407054040581 
Epoch:  31
2/16.8125 loss: 0.0037265519301096597 
Epoch:  31
3/16.8125 loss: 0.0033602953772060573 
Epoch:  31
4/16.8125 loss: 0.00317580453120172 
Epoch:  31
5/16.8125 loss: 0.002944354860422512 
Epoch:  31
6/16.8125 loss: 0.0027345974397446427 
Epoch:  31
7/16.8125 loss: 0.00260696055192966 
Epoch:  31
8/16.8125 loss: 0.0029440515079639023 
Epoch:  31
9/16.8125 loss: 0.0028201002511195838 
Epoch:  31
10/16.8125 loss: 0.0030187914380803704 
Epoch:  31
11/16.8125 loss: 0.002887096459744498 
Epoch:  31
12/16.8125 loss: 0.0028318424189749817 
Epoch:  31
13/16.8125 loss: 0.0027938730199821293 
Epoch:  31
14/16.8125 loss: 0.0026928597129881383 
Epoch:  31
15/16.8125 loss: 0.0030285085958894342 
Epoch:  31
16/16.8125 loss: 0.0029388538917855304 
Epoch:  32
0/16.8125 loss: 0.0012515936978161335 
Epoch:  32
1/16.8125 loss: 0.0016426232177764177 
Ep

Epoch:  40
11/16.8125 loss: 0.02566571839270182 
Epoch:  40
12/16.8125 loss: 0.02376939217524173 
Epoch:  40
13/16.8125 loss: 0.022378644049500247 
Epoch:  40
14/16.8125 loss: 0.020968000784826777 
Epoch:  40
15/16.8125 loss: 0.019712388322659535 
Epoch:  40
16/16.8125 loss: 0.018600893087055096 
Epoch:  41
0/16.8125 loss: 0.001389227109029889 
Epoch:  41
1/16.8125 loss: 0.011850698501802981 
Epoch:  41
2/16.8125 loss: 0.00816595236149927 
Epoch:  41
3/16.8125 loss: 0.006464726640842855 
Epoch:  41
4/16.8125 loss: 0.0053470124490559105 
Epoch:  41
5/16.8125 loss: 0.004641369528447588 
Epoch:  41
6/16.8125 loss: 0.004081867236111846 
Epoch:  41
7/16.8125 loss: 0.003671333790407516 
Epoch:  41
8/16.8125 loss: 0.003427163038092355 
Epoch:  41
9/16.8125 loss: 0.0032023388426750897 
Epoch:  41
10/16.8125 loss: 0.0031745104034515944 
Epoch:  41
11/16.8125 loss: 0.009762150739940504 
Epoch:  41
12/16.8125 loss: 0.009071809884447318 
Epoch:  41
13/16.8125 loss: 0.00848140484387321 
Epoch:  41


Epoch:  50
7/16.8125 loss: 0.0026874718387261964 
Epoch:  50
8/16.8125 loss: 0.00249388350898193 
Epoch:  50
9/16.8125 loss: 0.002400051616132259 
Epoch:  50
10/16.8125 loss: 0.0022606832102279773 
Epoch:  50
11/16.8125 loss: 0.0021533336548600346 
Epoch:  50
12/16.8125 loss: 0.0022305699441438685 
Epoch:  50
13/16.8125 loss: 0.0022063928855849163 
Epoch:  50
14/16.8125 loss: 0.002112075213032464 
Epoch:  50
15/16.8125 loss: 0.002095687021210324 
Epoch:  50
16/16.8125 loss: 0.002032319172833334 


In [282]:
# Evaluation
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
      
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()

        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [283]:
print("For: ",t,classification_report(test_y, bert_predicted,digits=4))


For:  Feminist Movement               precision    recall  f1-score   support

       False     0.7958    0.8306    0.8128       183
        True     0.3800    0.3276    0.3519        58

    accuracy                         0.7095       241
   macro avg     0.5879    0.5791    0.5823       241
weighted avg     0.6957    0.7095    0.7019       241



In [226]:
# from sklearn.metrics import roc_curve
# import sklearn
# from sklearn.metrics import f1_score

# fpr, tpr, threshold = roc_curve(y_test, y_predict)
# roc_auc = sklearn.metrics.auc(_fpr, _tpr)
# f1_score = f1_score(y_test, y_predict)
# print('AUROC: {}'.format(roc_auc), file=open("output.txt","a"))
# print('f1-score: {}'.format(f1_score), file=open("output.txt","a"))